In [1]:
!pip install optuna

  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11112 sha256=f427d94cbaff79ab07fd571248a2530a99947ee7685f7a2d00de46188228e81a
  Stored in directory: c:\users\itnes\appdata\local\pip\cache\wheels\7f\1a\65\84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


In [2]:
!pip install xgboost

In [4]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
df=pd.read_csv('Admission_Prediction.csv')

In [33]:
df['GRE Score']=df['GRE Score'].fillna(df['GRE Score'].mean())
df['TOEFL Score']=df['TOEFL Score'].fillna(df['TOEFL Score'].mean())
df['University Rating']=df['University Rating'].fillna(df['University Rating'])

In [34]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.000000,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.000000,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,316.558763,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.000000,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.000000,103.0,2.0,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
495,496,332.000000,108.0,5.0,4.5,4.0,9.02,1,0.87
496,497,337.000000,117.0,5.0,5.0,5.0,9.87,1,0.96
497,498,330.000000,120.0,5.0,4.5,5.0,9.56,1,0.93
498,499,312.000000,103.0,4.0,4.0,5.0,8.43,0,0.73


In [8]:
x=df.drop(columns=['Serial No.', 'Chance of Admit' ])

In [9]:
x

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337.0,118.0,4.0,4.5,4.5,9.65,1
1,324.0,107.0,4.0,4.0,4.5,8.87,1
2,NaN,104.0,3.0,3.0,3.5,8.00,1
3,322.0,110.0,3.0,3.5,2.5,8.67,1
4,314.0,103.0,2.0,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
495,332.0,108.0,5.0,4.5,4.0,9.02,1
496,337.0,117.0,5.0,5.0,5.0,9.87,1
497,330.0,120.0,5.0,4.5,5.0,9.56,1
498,312.0,103.0,4.0,4.0,5.0,8.43,0


In [14]:
y=df['Chance of Admit']

In [15]:
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit, Length: 500, dtype: float64

In [41]:
from sklearn.metrics import mean_squared_error
def objective(trial, data = x, target = y):
    train_x, test_x , train_y , test_y = train_test_split(data, target, test_size = 0.1, random_state = 30)
    param = {
        'tree_method':'gpu_hist',
        'lambda':trial.suggest_loguniform('lambda', 1e-4,10.0),
        'alpha':trial.suggest_loguniform('alpha',1e-4,10.0),
        'colsample_bytree' :trial.suggest_categorical('colsample_bytree', [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'subsample':trial.suggest_categorical('subsample', [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'learning_rate':trial.suggest_categorical('learning_rate', [.00001,.00003,.008,.02,.01,.1,10,20]),
        'n_estimators':30000,
        'max_depth': trial.suggest_categorical('max_depth',[3,4,5,6,7,8,9,10,11,12]),
        'random_state':trial.suggest_categorical('random_state',[10,20,30,2000,3454,243123]),
        'min_child_weight':trial.suggest_int('min_child_weight',1,200),
        'verbosity':3,
            }
    xgb_reg_model = xgb.XGBRFRegressor(**param)
    xgb_reg_model.fit(train_x, train_y, eval_set = [(test_x,test_y)], verbose = True)
    pred_xgb = xgb_reg_model.predict(test_x)
    rmse = mean_squared_error(test_y, pred_xgb)
    return rmsej

In [62]:
find_param=optuna.create_study(direction='minimize')
find_param.optimize(objective, n_trials=10)
find_param.best_trail.params


[I 2021-10-31 17:14:38,815] A new study created in memory with name: no-name-c1f46853-5614-4ef7-8392-6e849e9ed668
[W 2021-10-31 17:14:39,626] Trial 0 failed because of the following error: XGBoostError('[17:14:39] C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/gbm/gbtree.cc:588: Check failed: common::AllVisibleGPUs() >= 1 (0 vs. 1) : No visible GPU is found for XGBoost.')
Traceback (most recent call last):
  File "C:\Users\itnes\anaconda3\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-41-84b9060e50bf>", line 18, in objective
    xgb_reg_model.fit(train_x, train_y, eval_set = [(test_x,test_y)], verbose = True)
  File "C:\Users\itnes\anaconda3\lib\site-packages\xgboost\core.py", line 506, in inner_f
    return f(**kwargs)
  File "C:\Users\itnes\anaconda3\lib\site-packages\xgboost\sklearn.py", line 1533, in fit
    super().fit(**args)
  File "C:\Users\itnes\anaconda3\lib\site-packages\xgboost

[17:14:39] DEBUG: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/tree/updater_gpu_hist.cu:872: [GPU Hist]: Configure
[17:14:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:223: No visible GPU is found, setting `gpu_id` to -1


XGBoostError: [17:14:39] C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/gbm/gbtree.cc:588: Check failed: common::AllVisibleGPUs() >= 1 (0 vs. 1) : No visible GPU is found for XGBoost.

In [44]:
!NVIDIA-SMI

'NVIDIA-SMI' is not recognized as an internal or external command,
operable program or batch file.


In [35]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,500.000000,500.000000,485.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.103952,6.051338,1.146160,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,309.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,316.558763,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,324.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [45]:
    find_param.trails_dataframe()

AttributeError: 'Study' object has no attribute 'trails_dataframe'

In [46]:
optuna.visualisation

AttributeError: module 'optuna' has no attribute 'visualisation'

In [55]:
# classifiction with xgboost

In [49]:
new_df=pd.read_csv('winequality_red.csv')

In [50]:
new_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [52]:
new_df.describe

<bound method NDFrame.describe of       fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0               7.4             0.700         0.00             1.9      0.076   
1               7.8             0.880         0.00             2.6      0.098   
2               7.8             0.760         0.04             2.3      0.092   
3              11.2             0.280         0.56             1.9      0.075   
4               7.4             0.700         0.00             1.9      0.076   
...             ...               ...          ...             ...        ...   
1594            6.2             0.600         0.08             2.0      0.090   
1595            5.9             0.550         0.10             2.2      0.062   
1596            6.3             0.510         0.13             2.3      0.076   
1597            5.9             0.645         0.12             2.0      0.075   
1598            6.0             0.310         0.47             3.6      0.0

In [53]:
x=new_df.drop(columns='quality')

In [54]:
y=new_df.quality

In [60]:
def objective_classification(trail):
    train_x, test_x, train_y, teat_y=train_test_split(x,y, test_size=.2)
    param={
        'tree_method': 'gpu_hist',
        'verbosity': 3,
        'objective': "binary: logistics",
        'booster': trail.suggest_catogorical('booster', ['dart', 'gbtree', 'gblinear']),
        'lambda': trail.suggest_float('lambda', 1e-4, 1),
        'aplha': trail.suggest_float('alpha', 1e-4, 1),
        'subsample': trail.suggest_float('subsample', [.1, .2, .3, .4, .5]),
        'colsample_bytree': trail.suggest_float('colsample_bytree', .1, .5)
        
    }
    if param['booster'] in ['gbtree', 'dart']:
        param['gama'] : trail.suggest_float('gamma', 1e-3, 4)
        param['eta']: trail.suggest_float('eta', .001, 5)
    xgb_classification=xgb.XGBClassifier(**param)
    xgb_classification.fit(train_x, train_y, eval_set=[test_x, test_y])
    pred=xgb_classification.predict(test_x)
    accuracy=xgb_classification.score(test_y, pred)
    
    return accuracy
    

In [61]:
xgb_classification_optuna=optuna.create_study(direction='minimize')
xgb_classification_optuna.optimize(objective_classification, n_trials=10)

[I 2021-10-31 17:14:19,198] A new study created in memory with name: no-name-9b1d6e2b-095a-4467-88de-0b9cec74cadb
[W 2021-10-31 17:14:19,365] Trial 0 failed because of the following error: AttributeError("'Trial' object has no attribute 'suggest_catogorical'")
Traceback (most recent call last):
  File "C:\Users\itnes\anaconda3\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-60-9d60c68f2e57>", line 7, in objective_classification
    'booster': trail.suggest_catogorical('booster', ['dart', 'gbtree', 'gblinear']),
AttributeError: 'Trial' object has no attribute 'suggest_catogorical'


AttributeError: 'Trial' object has no attribute 'suggest_catogorical'